In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
    maxEventsToPrint = cms.untracked.int32(1),
    pythiaPylistVerbosity = cms.untracked.int32(1),
    
    pythiaHepMCVerbosity = cms.untracked.bool(False),
    comEnergy = cms.double(13600.),
    PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'pythia8PSweightsSettings',
                                    )
    )
)

ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator():
    dict = { '2022': pythia_fragment_CP5,
             '2022EE': pythia_fragment_CP5,
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2022", "2022EE"]
mX = ["500", "750", "1000", "1250", "1500", "1750", "2000", "2500", "3000", "4000"]
scenario = ["4", "10", "20", "50"]
tot_events1 = [1000000]*len(mX)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/gmilella12/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/13p6TeV/TopPhilicZp/ttZp'
    t_datasetname_year = 'TTZprimetoTT_M-{massX}_Width{scenario}_TuneCP5_13p6TeV_madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/RunIII/13p6TeV/el8_amd64_gcc10/MadGraph5_aMCatNLO/TTZprimeToTT/TopPhilic_ttzp_13p6TeV_nodecay_m{massX}_relwidth{scenario}_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    
    dataset_names_year = []
    gp_ggf_year = []

    for mx in mX:
        for scena in scenario:
            d = {"massX": mx, "scenario": scena}
            tmp_example_dict = t_example1
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict = dataset_names_year
            tmp_gridpack_dict = gp_ggf_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator()
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('ttzprimetott' + '_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        #if year == "2022":
        #    pr = 5648
        #if year == "2022EE":
        #    pr = 5224
            
        k=0
        for i in range(len(mX)):
            for j in range(len(scenario)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.9.18"
                dataset_name = dataset_names[year][k]
                if year == "2022":
                    events = round(tot_events1[i]*0.22)
                    #prepid = 'B2G-Run3Summer22wmLHEGS-0' + str(pr)
                elif year == "2022EE":
                    events = round(tot_events1[i]*0.78)
                    #prepid = 'B2G-Run3Summer22EEwmLHEGS-0' + str(pr)
                
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + " Pythia8"
                mcdb_id = '0'
                time = '0.01'
                size = '30'
                csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                #pr = pr+1   
                k=k+1